In [1]:
import numpy as np
import matplotlib.pyplot as plt


from copy import deepcopy

import gamspec

In [2]:
# 用于寻峰的分支
sav = gamspec.SavitzkySmoother(3, 3)
convol = gamspec.SecondConvolutionPeakSearcher()
fpeak = gamspec.Flow(gamspec.Pipe([sav, convol]), 0, 0, 1, 1)

# 用于平滑的分支
wavelet = gamspec.TranslationInvarianceWaveletSmoother('dmey', 'quadratic-soft', order=3)
fsmooth = gamspec.Flow(wavelet, 0, 0, 1, 0) 

# 两个分支合并
def combine(spec1, spec2):
    spec1.regions = deepcopy(spec2.regions)
    return spec1
comb = gamspec.Combinor(2, func=combine)
fcomb = gamspec.Flow(comb, 1, [0, 1], 2, 0)

# 剥谱
strp = gamspec.OtherOperator.AdaptiveSNIPStripper(baseline=False)
fstrip = gamspec.Flow(strp, 2, 0, 3, 0)

# 拟合，计算峰面积
fit = gamspec.RegionPeakFitter(10, equal_width=False, baseline=False)
ffit = gamspec.Flow(fit, 3, 0, 4, 0)

pipenet = gamspec.PipeNet([fpeak, fsmooth, fcomb, fstrip, ffit])

使用numpy产生随机数然后绘制直方图得到的高斯分布单峰。无本底。预设峰位置50，标准差1.3

In [3]:
# np.random.seed(1)

# signal = np.random.normal(loc=50, scale=1.3, size=1000)
# histo = np.histogram(signal, bins=100, range=(0,100))[0]
# spec = gamspec.Spectrum(histo)

# pipenet([spec])
# pipenet.get_node(4)[0].export_to_excel('test.xlsx')

使用numpy产生随机数然后绘制直方图得到的高斯分布单峰。无本底。预设峰位置50，标准差1.3，固定添加高度为100的本底后再进行正态扰动											

In [4]:
np.random.seed(1)

signal = np.random.normal(loc=50, scale=4, size=1000)
histo = np.histogram(signal, bins=100, range=(0,100))[0]
histo = histo + 100
print(histo.max()-100)
histo = np.random.normal(loc=histo, scale=histo**0.5)
spec = gamspec.Spectrum(histo)


114


In [8]:
nodes = pipenet([spec])
nodes[4][0].export_to_pandas()

GAMUT: Start running PipeNet
|Execute: Node 0|[0] >> SavitzkySmoother[O3W7]>>SecondConvolutionSearcher[F20] >> Node 1|[1]
|Execute: Node 0|[0] >> TIWaveletSmoother[O3MqTv] >> Node 1|[0]
|Execute: Node 1|[0, 1] >> Combinor >> Node 2|[0]
|Execute: Node 2|[0] >> AdaptiveSNIPStripper >> Node 3|[0]
|Execute: Node 3|[0] >> RegionPeakFitter >> Node 4|[0]
Fitting Region: 39~70, NPeaks=1, time=Wed Apr 10 16:36:21 2024
Finish Fitting Region: time=Wed Apr 10 16:36:21 2024


,left,right,energy,location,height,stderr,area,sig_area2,fitness
0,39,70,50,50,105.919719,4.85744,1289.657064,94.480453,0.888374


In [9]:
pipenet.get_node(3)[0].plot()
pipenet.get_node(1)[0].plot()
pipenet.get_node(0)[0].plot()
plt.legend()

AttributeError: 'PipeNet' object has no attribute 'get_node'